In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [16]:
loader = PyPDFLoader("Documents/Building-Soils-for-Better-Crops.pdf")

In [17]:
pages = loader.load_and_split()

In [18]:
pages[:3]

[Document(page_content='BUILDING SOILS FOR BETTER CROPS: ECOLOGICAL MANAGEMENT FOR HEALTHY SOILS3\nChapter 1\nHEALTHY SOILS\nAll over the country [some soils are] worn out, depleted, exhausted, almost dead.  \nBut here is comfort: These soils possess possibilities and may be restored to high  \nproductive power, provided you do a few simple things. \n—C.W. BURKETT, 1907 \nIt should come as no surprise that many cultures \nhave considered soil central to their lives. After all, peo-\nple were aware that the food they ate grew from the soil. Our ancestors who first practiced agriculture must have been amazed to see life reborn each year when seeds placed in the ground germinated and then grew to matu -\nrity. In the Hebrew Bible, the name given to the first man,\n Adam, is the masculine version of the word “earth” \nor “soil” (adama). The name for the first woman, Eve (or Hava in Hebrew), comes from the word for “living.” Soil and human life were considered to be intertwined. A particula

In [19]:
for i in range(3):
    print(pages[i].metadata)

{'source': 'Documents/two_chapters.pdf', 'page': 0}
{'source': 'Documents/two_chapters.pdf', 'page': 1}
{'source': 'Documents/two_chapters.pdf', 'page': 1}


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pages)

In [21]:
print(f"{len(pages)} vs {len(documents)}")

23 vs 84


In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
open_ai_key = os.getenv("OPENAI_API_KEY")

In [23]:
from langchain.embeddings import OllamaEmbeddings
embeddings=OllamaEmbeddings()

In [24]:
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(documents, embeddings)

In [33]:
vector.save_local("faiss_index_full_book")

In [35]:
vector = FAISS.load_local("faiss_index_full_book", embeddings, allow_dangerous_deserialization=True)


In [25]:
from langchain_groq import ChatGroq
groq_api_key = os.environ['GROQ_API_KEY']
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="mixtral-8x7b-32768")

In [26]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [28]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [29]:
prompt=("What all services soil provides")
response=retrieval_chain.invoke({"input":prompt})
print(response['answer'])

Soil provides several essential services, including:

1. Supporting plant growth: Soil provides a physical foundation for plants to grow in, anchoring their roots and supplying water and nutrients.
2. Recycling nutrients: Soil acts as a natural recycling system, breaking down organic matter and releasing nutrients that plants need to grow.
3. Filtering and buffering: Soil can filter and buffer pollutants, helping to protect groundwater and surface water from contamination.
4. Carbon sequestration: Soil can store large amounts of carbon, helping to mitigate climate change.
5. Habitat: Soil provides a habitat for a diverse range of organisms, including bacteria, fungi, and small animals, which contribute to soil health and functioning.
6. Supporting biodiversity: Soil supports a wide range of plant and animal species, contributing to biodiversity and ecosystem health.

In the provided context, the emphasis is on the role of soil in supporting plant growth and nutrient cycling. The text h

In [32]:
prompt=("what all maesure i can take to have healthy soils which are relatively easy to perform")
response=retrieval_chain.invoke({"input":prompt})
print(response['answer'])



Here are some measures you can take to have healthy soils that are relatively easy to perform:

1. Add Organic Matter: Incorporating organic matter such as compost, leaf litter, or well-rotted manure can significantly improve soil health. Organic matter helps to improve soil structure, increase water-holding capacity, and provide nutrients for plants.

2. Practice Crop Rotation: Rotating crops can help to break disease cycles, reduce pest pressure, and improve soil structure. By varying the types of crops grown in a particular area, you can also help to maintain soil nutrient levels.

3. Minimize Tillage: Tillage can disrupt soil structure, lead to erosion, and release carbon dioxide into the atmosphere. Minimizing tillage can help to maintain soil structure, increase soil organic matter, and reduce erosion.

4. Use Cover Crops: Cover crops can help to protect soil from erosion, add organic matter, and suppress weeds. They can also help to improve soil structure and increase soil biodi

In [40]:
prompt=("what are all possible questions I can ask from you?")
response=retrieval_chain.invoke({"input":prompt})
print(response['answer'])



Here are some potential questions that can be asked based on the provided context:

1. What is the concept of risk in relation to weather extremes and their impact on farms and communities?
2. How does soil health affect the likelihood of wind erosion?
3. What are the three aspects of vulnerability to weather-related challenges?
4. How does soil aggregation affect the infiltration capacity of the soil?
5. What are the effects of soil degradation on excessive runoff and flooding?
6. How do soils help to purify wastes from septic system drain fields?
7. What is the role of soil organic matter in relation to atmospheric carbon?
8. How do soils provide habitats for diverse organisms, including those that produce antibiotics and improve soil structure?
9. Who are the authors of the studies and publications mentioned in the context?
10. What are the figures in the context illustrating or depicting?
11. How does vegetation affect soil degradation and runoff?
12. What are the different process

In [31]:
for i, doc in enumerate(response["context"]):
    print(doc.page_content)
    print("--------------------------------")

its or the Haber-Bosch process for producing nitrogen fertilizers.
 But the need to replace the organic matter 
(carbon) was mostly ignored until recently. 
The organic matter in the soil is more complex and 
plays many important roles in soils that we will discuss in Chapter 2. Not only does it store and supply nutrients and energy for organisms, it also helps form aggregates when mineral and organic particles clump together. When it is made up of large amounts of different-sized aggregates, the soil contains more spaces for storing water and allowing gas exchange, as oxygen enters for use by plant roots and by soil organisms and the carbon dioxide produced by organisms leaves the soil. So in summary, the mineral particles and pore spaces form the basic structure of the soil, but the organic matter is mostly what makes it fertile.
WHAT KIND OF SOIL DO YOU WANT? 
Farmers sometimes use the term soil health  to describe
--------------------------------
Soil degradation caused significant

In [46]:
response["context"]

[Document(page_content='In devel -\noped societies nutritional deficiencies are rare because people obtain food from diverse sources.\n For example, \nregional soil selenium deficiency does not impact people  \nwhen they also eat nuts from other regions. (In developed  \nsocieties, the concern is increasingly about unhealthy diet choices and the affordability of healthy food.)\nHumans also benefi\nt from organic plant compounds', metadata={'source': 'Documents/HealthySoils.pdf', 'page': 8}),
 Document(page_content='the nurturing process of converting a degraded or low-quality soil into a truly high-quality one requires understanding, thought and significant actions. It is a process that mirrors the building of soil through natural processes where plants and organic matter are key elements. This is also true for maintaining or \nFigure 1.2. Agricultural soil (left) and natural soil (grassland; right) from \nadjacent sites in the U.S. Great Plains. Agricultural soil has lower soil organi